## Техническое описание кода:

**Функциональность:**

1. **Взаимодействие с LLM:**
    - Класс `BaseAgent` инкапсулирует логику взаимодействия с LLM (конкретно, моделью `gpt-4o-mini` через OpenAI API). 
    - Он принимает системный промпт при инициализации и предоставляет метод `__call__` для отправки сообщений модели и получения ответов.

2. **Оптимизация промпта:**
    - Класс `PromptOptimizer` отвечает за оптимизацию промпта, используя экземпляр `BaseAgent` для связи с LLM. 
    - Он принимает на вход:
        -  исходный промпт, 
        -  данные для обучения (опционально), 
        -  параметры оптимизации (скорость обучения, количество итераций, размер пучка для Монте-Карло, параметры градиентного спуска, критерии оценки в формате JSON).

3. **Генерация кандидатов (`generate_candidates`)**:
    - Создает список потенциально улучшенных промптов на основе текущего, используя два подхода:
        - **Метод Монте-Карло (`generate_monte_carlo_candidate`)**: Генерирует перефразированные версии промпта, запрашивая у LLM "перефразировать, сохраняя смысл".
        - **Градиентный спуск (`generate_gradient_candidate`)**: 
            - Получает ответ LLM на текущий промпт.
            - Вызывает `generate_gradient()` для анализа ответа и получения текстового описания потенциальных улучшений (градиента).
            - Запрашивает у LLM несколько (`steps_per_gradient`) улучшенных вариантов промпта, основываясь на сгенерированном градиенте.

4. **Оценка кандидатов (`evaluate_candidates`)**:
    - Оценивает каждый сгенерированный промпт-кандидат, вызывая `evaluate_response()` и возвращая список оценок.

5. **Оценка ответа (`evaluate_response`)**:
    - **Ключевая функция, определяющая качество ответа LLM на данный промпт.**
    - Анализирует ответ на соответствие критериям, заданным в `evaluation_criteria`:
        - Для каждого критерия вызывает `check_criterion()`, чтобы проверить, выполняется ли условие.
        - Начисляет или снимает баллы в зависимости от важности (`weight`) и обязательности (`required`) критерия.
    - Дополнительно рассчитывает метрики ROUGE и BLEU, сравнивая ответ с исходным промптом, и включает их в общую оценку с заданными весами.
    - Возвращает нормализованную оценку в диапазоне [-1, 1].

6. **Проверка критерия (`check_criterion`)**:
    - Принимает на вход ответ LLM и строку `criterion_check`, содержащую код Python для проверки.
    - Выполняет этот код, передавая `response` как контекст, что позволяет задавать **произвольные критерии оценки** на языке Python.
    - Возвращает True, если условие выполнено, и False в противном случае.

7. **Выбор лучшего кандидата (`select_best_candidate`)**:
    - Использует алгоритм UCB для выбора наилучшего кандидата из списка, балансируя между исследованием новых кандидатов и эксплуатацией уже известных хороших.
    - Учитывает не только среднюю оценку кандидата, но и дисперсию оценок, что позволяет делать более обоснованный выбор в условиях неопределенности.

8. **Генерация градиента (`generate_gradient`)**:
    - **Инновационная функция, автоматизирующая процесс определения критериев оценки и генерации градиента для улучшения промпта.**
    - Отправляет LLM запрос с текущим промптом и просьбой:
        - Определить наиболее важные критерии оценки для этого промпта.
        - Сгенерировать для каждого критерия текстовое описание возможных улучшений (градиент).
    - Ожидает ответ в строго определенном формате JSON, содержащем список критериев с их описанием, важностью, обязательностью и кодом проверки.
    - Анализирует ответ LLM на соответствие сгенерированным критериям.
    - Формирует итоговый текстовый градиент, перечисляя невыполненные критерии и предлагаемые LLM улучшения.

**Механизмы:**

- **Метод Монте-Карло:** Используется для стохастического поиска в пространстве промптов, генерируя случайные вариации.
- **Градиентный спуск:**  Направляет поиск в сторону улучшения промпта, основываясь на анализе ответов LLM и сгенерированных градиентах.
- **Алгоритм UCB:**  Обеспечивает баланс между исследованием новых кандидатов и использованием уже известных хороших, ускоряя сходимость оптимизации.
- **Динамическая генерация критериев оценки:** Делегирует LLM задачу определения критериев оценки, адаптируя процесс оптимизации к конкретным промптам и задачам.
- **Метрики ROUGE и BLEU:** Используются для оценки качества сгенерированного текста путем сравнения его с исходным промптом. ROUGE измеряет степень перекрытия n-грамм между текстами, а BLEU оценивает точность перевода, сравнивая сгенерированный текст с несколькими референтными переводами. 



---

### **Математическая формализация и пояснение механизмов**

#### **Метод Монте-Карло**

#### **Определение**

**Метод Монте-Карло** — это стохастический метод, который использует случайные выборки для моделирования и решения математических и физических задач. В контексте поиска в пространстве промптов, метод Монте-Карло генерирует случайные вариации промптов и оценивает их эффективность с целью нахождения оптимального или приближенного решения.

#### **Механизм работы алгоритма**

Процесс метода Монте-Карло можно разделить на несколько основных шагов:

1. **Генерация случайных промптов.** В этом шаге случайно выбираются или варьируются параметры, которые влияют на формирование промпта.
2. **Оценка промптов.** Каждый сгенерированный промпт оценивается по заранее определенным метрикам или критериям (например, по качеству сгенерированного текста).
3. **Агрегация результатов.** На основе множества испытаний рассчитывается средняя или интегральная метрика, характеризующая эффективность определенных параметров.
4. **Анализ результатов.** Изучение распределения результатов позволяет сделать выводы о том, какие параметры или их комбинации работают лучше всего.

#### **Математическая формализация**

Предположим, что мы оцениваем функцию $f(x)$ на основе случайных выборок $x_1, x_2, \dots, x_N$, которые следуют определенному распределению вероятностей $P(x)$. Оценка ожидаемого значения $E[f(x)]$ функции $f(x)$ с помощью метода Монте-Карло вычисляется как:

$$E[f(x)] \approx \frac{1}{N} \sum_{i=1}^{N} f(x_i)$$

где $N$ — количество случайных выборок.

#### **Вывод**

Метод Монте-Карло — это мощный и гибкий инструмент, который может быть использован для решения задач оптимизации в случаях, когда аналитическое решение невозможно или трудно найти. В контексте генерации и оптимизации промптов этот метод помогает находить лучшие вариации за счет случайного исследования пространства возможных решений.

---

#### **Алгоритм UCB**

#### **Определение**

**Upper Confidence Bound (UCB)** — это алгоритм, предназначенный для решения задачи многорукого бандита, который балансирует между исследованием новых вариантов (exploration) и использованием известных хороших вариантов (exploitation). В контексте оптимизации промптов, UCB позволяет ускорить процесс нахождения оптимального промпта, обеспечивая быстрый переход от исследования новых вариантов к использованию найденных эффективных решений.

#### **Механизм работы алгоритма**

Алгоритм UCB работает следующим образом:

1. **Инициализация.** В начале каждому промпту присваивается начальное значение оценки.
2. **Выбор промпта.** Для каждого промпта рассчитывается доверительный интервал, и выбирается тот, у которого верхняя граница доверительного интервала (Upper Confidence Bound) максимальна.
3. **Обновление оценки.** После выбора и оценки промпта, его значение обновляется на основе полученного результата.
4. **Повторение процесса.** Шаги 2 и 3 повторяются до достижения сходимости, при этом со временем алгоритм все больше использует наиболее успешные промпты.

#### **Математическая формализация**

Для каждого промпта $i$ на шаге $t$ UCB рассчитывается следующим образом:

$$a_i(t) = \bar{x}_i(t) + c \sqrt{\frac{2 \ln t}{n_i(t)}}$$

где:
- $\bar{x}_i(t)$ — среднее значение награды для промпта $i$ до момента $t$,
- $n_i(t)$ — количество раз, когда промпт $i$ был выбран до момента $t$,
- $c$ — параметр, регулирующий степень исследования (exploration),
- $t$ — текущее количество шагов.

#### **Вывод**

Алгоритм UCB эффективно балансирует между исследованием новых вариантов и использованием известных успешных вариантов. В задачах оптимизации промптов UCB может существенно сократить время, необходимое для нахождения оптимального решения, благодаря своим адаптивным свойствам.

---

#### **Метрики ROUGE и BLEU**

#### **Определение**

**ROUGE** (Recall-Oriented Understudy for Gisting Evaluation) и **BLEU** (Bilingual Evaluation Understudy) — это метрики, используемые для оценки качества текста, сгенерированного алгоритмами на основе исходного текста (референсного промпта). ROUGE измеряет степень перекрытия n-грамм между сгенерированным текстом и референсом, в то время как BLEU оценивает точность перевода, сравнивая сгенерированный текст с одним или несколькими референтными переводами.

#### **Механизм работы алгоритма**

#### **ROUGE**
1. **Разбиение текста на n-граммы.** Оба текста (сгенерированный и референсный) разбиваются на n-граммы — последовательности из n слов.
2. **Подсчет совпадений.** Подсчитывается количество совпадений n-грамм между двумя текстами.
3. **Оценка перекрытия.** Рассчитываются различные метрики, такие как ROUGE-N, ROUGE-L, которые отражают степень перекрытия n-грамм, длины последовательностей слов и т.д.

#### **BLEU**
1. **Разбиение текста на n-граммы.** Текст и референсный перевод разбиваются на n-граммы.
2. **Подсчет совпадений.** Подсчитывается количество совпадений n-грамм между сгенерированным текстом и каждым референсным переводом.
3. **Корректировка длины.** Вводится штраф за длину, чтобы учитывать, что слишком короткий текст может получить высокую оценку за совпадение n-грамм, но не быть точным переводом.
4. **Сводка результатов.** Вычисляется среднее значение точности для всех n-грамм с учетом штрафа за длину.

#### **Математическая формализация**

#### **ROUGE-N**
Метрика ROUGE-N для n-грамм вычисляется как:

$$\text{ROUGE-N} = \frac{\sum_{s \in \text{ref}} \sum_{n\text{-gram} \in s} \min(\text{Count}_\text{match}(n\text{-gram}), \text{Count}_\text{gen}(n\text{-gram}))}{\sum_{s \in \text{ref}} \sum_{n\text{-gram} \in s} \text{Count}_\text{ref}(n\text{-gram})}$$

#### **BLEU**
Метрика BLEU вычисляется как:

$$\text{BLEU} = BP \cdot \exp \left( \sum_{n=1}^{N} w_n \log p_n \right)$$

где:
- $BP$ — штраф за длину (brevity penalty),
- $p_n$ — точность для n-грамм,
- $w_n$ — весовая коэффициента для n-грамм.

#### **Вывод**

Метрики ROUGE и BLEU являются стандартными инструментами для оценки качества сгенерированного текста в задачах обработки естественного языка. ROUGE подчеркивает степень покрытия текстовых сегментов, а BLEU — точность и адекватность перевода. Эти метрики помогают объективно оценивать, насколько сгенерированный текст соответствует ожиданиям на основе исходных данных.


In [1]:
# Импорт стандартных библиотек
import json
import logging
import uuid
from pathlib import Path

import numpy as np
import openai
import rouge
import sacrebleu

# Интерфейс для взаимодействия с LLM
from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage

from typing import List

# Импорт внутренних библиотек
from back.file_manager import FileManager

---
### Test 1 (Chat-GPT)

In [7]:
class BaseAgent:
    """
    Description:
        Класс для взаимодействия с LLM через OpenAI API.

    Attributes:
        system_prompt: Начальный системный промпт для модели.
    """
    def __init__(self, system_prompt: str):
        """
        Description:
            Инициализация класса BaseAgent.

        Args:
            system_prompt: Начальный системный промпт для модели.
        """
        self.system_prompt = system_prompt

    def __call__(self, messages: list) -> str:
        """
        Description:
            Выполнение вызова к LLM с заданными сообщениями.

        Args:
            messages: Список сообщений для отправки в LLM.

        Returns:
            Ответ модели в виде строки.
        """
        response = openai.chat.completions.create(
            model="gpt-4o-mini", 
            messages=[
                {"role": "system", 
                 "content": self.system_prompt},
                *messages
            ]
        ).choices[0].message.content
        return response

---
### Test 2 (TGI - Llama 3.1 70B)

In [4]:
# class BaseAgent:
#     """
#     Description:
#         Класс для взаимодействия с моделью ChatOpenAI.
#     """
    
#     def __init__(self, system_prompt: str):
#         """
#         Description:
#             Инициализирует экземпляр CustomChatModel.
            
#         Args:
#             system_prompt: Начальный системный промпт для модели.
#         """
#         self.system_prompt = system_prompt
        
#         self.client = ChatOpenAI(
#             base_url=os.getenv("TGI_URL"),
#             api_key="-",
#             model=os.getenv("MODEL_NAME"),
#             temperature=0.1,
#             n=10,
#             top_p=0.9,
#             max_tokens=4096,
#             streaming=False,
#             verbose=True,
#         )
    
#     def __call__(self, messages: List[BaseMessage]) -> str:
#         """
#         Description:
#             Генерирует ответ от модели на основе предоставленных сообщений.

#         Args:
#             messages (List[BaseMessage]): Список сообщений для отправки модели.

#         Returns:
#             str: Ответ от модели в виде строки.
#         """
#         messages = [
#             SystemMessage(content = self.system_prompt),
#             HumanMessage(content = messages)
#         ]
        
#         # Вызов API
#         response = self.client.invoke(messages)
        
#         return response.content

In [5]:
class PromptOptimizer:
    """
    Description:
        Класс для оптимизации промпта с использованием методов Монте-Карло и градиентного спуска.

    Attributes:
        llm: Экземпляр класса BaseAgent для взаимодействия с LLM.
        prompt: Исходный промпт для оптимизации.
        num_iterations: Количество итераций оптимизации.
        beam_size: Размер выборки для поиска лучшего кандидата.
        steps_per_gradient: Количество шагов на градиент.
        evaluation_criteria: Конфигурация для оценки ответов модели.
    """
    def __init__(self,
                 llm: BaseAgent,
                 initial_prompt: str,
                 num_iterations: int = 1,
                 beam_size: int = 4,
                 steps_per_gradient: int = 2,
                 evaluation_criteria: str = '{}'):
        """
        Description:
            Инициализация класса PromptOptimizer.

        Args:
            llm: Экземпляр класса BaseAgent для взаимодействия с LLM.
            initial_prompt: Исходный промпт для оптимизации.
            num_iterations: Количество итераций оптимизации.
            beam_size: Размер выборки для поиска лучшего кандидата.
            steps_per_gradient: Количество шагов на градиент.
            evaluation_criteria: Конфигурация для оценки ответов модели в формате JSON.
        """
        self.llm = llm
        self.prompt = initial_prompt
        self.num_iterations = num_iterations
        self.beam_size = beam_size
        self.steps_per_gradient = steps_per_gradient
        self.evaluation_criteria = json.loads(evaluation_criteria)

    def optimize(self) -> str:
        """
        Description:
            Основной метод оптимизации промпта.

        Returns:
            Оптимизированный промпт.
        """
        for i in range(self.num_iterations):
            print(f"[INFO] === Iteration {i+1}/{self.num_iterations} ===")
            # Генерируем кандидатов на основе текущего промпта
            candidates = self.generate_candidates()
            # Оцениваем каждого кандидата
            candidate_scores = self.evaluate_candidates(candidates, self.evaluation_criteria)
            # Выбираем лучшего кандидата по результатам оценки
            best_candidate_idx = self.select_best_candidate(candidate_scores)
            # Обновляем текущий промпт лучшим кандидатом
            self.prompt = candidates[best_candidate_idx]
            print(f"[INFO] Best prompt at iteration {i+1}: {self.prompt}\n")
        return self.prompt

    def generate_candidates(self) -> list:
        """
        Description:
            Генерация списка кандидатов на основе исходного промпта.

        Returns:
            Список сгенерированных кандидатов.
        """
        candidates = [self.prompt]
        
        # Генерация кандидатов с использованием метода Монте-Карло
        for _ in range(self.beam_size):
            monte_carlo_candidate = self.generate_monte_carlo_candidate(self.prompt)
            candidates.append(monte_carlo_candidate)
        
        # Генерация кандидатов на основе градиентного спуска
        for _ in range(self.beam_size):
            gradient_candidate = self.generate_gradient_candidate(self.prompt)
            candidates.append(gradient_candidate)
        
        return candidates

    def generate_monte_carlo_candidate(self, prompt: str) -> str:
        """
        Description:
            Генерация кандидата с использованием метода Монте-Карло.

        Args:
            prompt: Исходный промпт.

        Returns:
            Сгенерированный промпт.
        """
        paraphrased_prompt = self.llm(messages=[{"role": "user", "content": f"Перефразируй следующий текст, сохраняя смысл:\n{prompt}"}])
        return paraphrased_prompt

    def generate_gradient_candidate(self, prompt: str) -> str:
        """
        Description:
            Генерация кандидата с использованием градиентного подхода.

        Args:
            prompt: Исходный промпт.

        Returns:
            Сгенерированный промпт с улучшениями.
        """
        # Получаем ответ от LLM на исходный промпт
        response = self.llm(messages=[{"role": "user", "content": prompt}])

        # Генерируем текстовые улучшения на основе анализа ответа LLM
        gradient = self.generate_gradient(prompt, response)

        # Запрашиваем у LLM улучшенные версии промпта на основе предложенных улучшений
        improved_prompt = self.llm(messages=[{"role": "user", "content": f"""
            Я пытаюсь написать промпт для ИИ-помощника.
            Мой текущий промпт:
            ```
            {prompt}
            ```
            Но он не идеален: {gradient}
            Учитывая вышеизложенное, я написал {self.steps_per_gradient} различных улучшенных промпта.
            Каждый промпт заключен в ```:
            """}])
        
        return improved_prompt

    def evaluate_candidates(self, candidates: list, evaluation_criteria: dict) -> list:
        """
        Description:
            Оценка кандидатов на основе заданных критериев.

        Args:
            candidates: Список кандидатов.
            evaluation_criteria: Словарь с критериями оценки.

        Returns:
            Список оценок для каждого кандидата.
        """
        print("[INFO] Evaluating candidates...")
        
        candidate_scores = []

        for candidate in candidates:
            # Получаем ответ от LLM на каждый кандидатский промпт
            response = self.llm(messages=[{"role": "user", "content": candidate}])
            # Оцениваем ответ на основе заданных критериев
            score = self.evaluate_response(response, evaluation_criteria)
            candidate_scores.append(score)
        
        print(f"[INFO] Candidate scores: {candidate_scores}")
        return candidate_scores

    def evaluate_response(self, response: str, evaluation_criteria: dict) -> float:
        """
        Description:
            Оценивает ответ LLM на основе заданных критериев, переданных в конфигурации.

        Args:
            response: Ответ модели.

        Returns:
            Оценка (число), представляющая качество ответа.
        """
        score = 0
        total_possible_score = 0
        
        for criterion in evaluation_criteria.get("criteria", []):
            criterion_name = criterion.get("name")
            criterion_required = criterion.get("required", False)
            criterion_check = criterion.get("check", "True")
            criterion_weight = criterion.get("weight", 1)
            total_possible_score += criterion_weight

            try:
                # Проверяем выполнение условия для критерия
                criterion_met = self.check_criterion(response, criterion_check)
                print(f"[DEBUG] Evaluating criterion '{criterion_name}': Met = {criterion_met}, Required = {criterion_required}, Weight = {criterion_weight}")
                
                if criterion_met:
                    score += criterion_weight
                elif criterion_required:
                    # Если критерий обязателен и не выполнен, снижаем итоговый балл
                    score -= criterion_weight
                else:
                    print(f"[WARN] Criterion '{criterion_name}' not met, but not required.")
            except Exception as e:
                print(f"[ERROR] Error evaluating criterion '{criterion_name}': {e}")
        
        # Вычисление метрик ROUGE и BLEU
        rouge_scorer = rouge.Rouge()
        rouge_scores = rouge_scorer.get_scores(response, self.prompt)[0]
        print(f"[INFO] ROUGE scores: {rouge_scores}")

        bleu_score = sacrebleu.corpus_bleu([response], [[self.prompt]]).score
        print(f"[INFO] BLEU score: {bleu_score}")

        # Включение ROUGE и BLEU в итоговую оценку
        rouge_weight = evaluation_criteria.get("rouge_weight", 0.2)
        bleu_weight = evaluation_criteria.get("bleu_weight", 0.2)

        total_possible_score += rouge_weight + bleu_weight
        score += rouge_scores['rouge-l']['f'] * rouge_weight
        score += bleu_score / 100 * bleu_weight

        # Нормализуем оценку в диапазоне от -1 до 1
        normalized_score = score / total_possible_score if total_possible_score > 0 else 0
        print(f"[INFO] Total score for the response: {score} / {total_possible_score} (Normalized: {normalized_score})")
        print('=' * 101)

        return max(min(normalized_score, 1), -1)  # Ограничиваем результат в диапазоне [-1, 1]


    def check_criterion(self, response: str, criterion_check: str) -> bool:
        """
        Description:
            Проверка ответа LLM на соответствие критерию.

        Args:
            response: Ответ LLM.
            criterion_check: Строка с условием проверки.

        Returns:
            True, если ответ соответствует критерию, иначе False.
        """
        # Формируем запрос к LLM, чтобы она проверила соответствие ответа критерию
        prompt = f"""
        Вход: 
        Ответ: \"{response}\"
        Критерий: \"{criterion_check}\"

        Вопрос: Выполняется ли критерий для данного ответа? Ответь 'True', если да, и 'False', если нет. 
        """
        
        # Получаем ответ от LLM на запрос
        llm_response = self.llm(messages=[{"role": "user", "content": prompt}])
        
        # Обрабатываем ответ модели
        llm_response = llm_response.strip().lower()
        
        # Проверяем, является ли ответ True или False
        if llm_response in ["true", "false"]:
            return llm_response == "true"
        else:
            print(f"[ERROR] Unexpected LLM response for criterion check: {llm_response}")
            return False

    def select_best_candidate(self, candidate_scores: list) -> int:
        """
        Description:
            Реализация алгоритма UCB для выбора лучшего кандидата.

        Args:
            candidate_scores: Оценки кандидатов.

        Returns:
            Индекс лучшего кандидата.
        """
        print("[INFO] Selecting the best candidate using UCB...")

        N = len(candidate_scores)       # Общее количество кандидатов
        T = np.arange(1, N + 1)         # Массив для отслеживания количества испытаний каждого кандидата
        Q = np.array(candidate_scores)  # Оценки кандидатов

        # Вычисление дисперсии оценок кандидатов
        variance = np.var(Q)
        print(f"[DEBUG] Variance of candidate scores: {variance}")

        # Расчет UCB с учетом дисперсии
        ucb_values = Q + np.sqrt(2 * np.log(T) / (T + 1e-5)) + variance

        print(f"[DEBUG] UCB values: {ucb_values}")

        # Выбор индекса кандидата с максимальным значением UCB
        best_idx = np.argmax(ucb_values)
        
        print(f"[INFO] Selected best candidate index: {best_idx}")
        
        return best_idx  # Возвращаем индекс лучшего кандидата

    def generate_gradient(self, prompt: str, response: str) -> str:
        """
        Description:
            Генерация градиента на основе динамического анализа ответа LLM и критериев оценки, заданных для конкретного промпта.

        Args:
            prompt: Исходный промпт.
            response: Ответ LLM на этот промпт.

        Returns:
            Текстовый градиент, указывающий на возможные улучшения.
        """
        # Шаг 1: Запрашиваем у LLM критерии оценки и градиенты для данного промпта
        evaluation_criteria_json = self.llm(messages=[{
            "role": "user",
            "content": f"""
                Мне нужно оценить ответ модели на следующий промпт:
                ```
                {prompt}
                ```
                В зависимости от этого промпта, определи какие критерии оценки наиболее важны для ответа модели. 
                Учти, что критерии должны быть основаны на содержании промпта, его цели, ожидаемом результате и других релевантных факторах. 

                Определи критерии оценки этого промпта и представь их в формате JSON.
                Также сформируй список возможных улучшений (градиентов) для каждого критерия в формате JSON.
                Пример ответа:
                {{
                    "criteria": [
                        {{
                            "name": "include_math",
                            "required": true,
                            "check": "response.count('math') > 0",
                            "gradient": "Добавьте математические формулы для лучшего объяснения.",
                            "weight": 0.3
                        }},
                        {{
                            "name": "latex_formatting",
                            "required": true,
                            "check": "response.count('\\$') > 0",
                            "gradient": "Убедитесь, что формулы правильно отформатированы в LaTeX.",
                            "weight": 0.3
                        }},
                        {{
                            "name": "clarity_and_conciseness",
                            "required": true,
                            "check": "len(response.split()) < 100",
                            "gradient": "Обеспечьте ясность и краткость в объяснениях.",
                            "weight": 0.2
                        }},
                        {{
                            "name": "detailed_explanation",
                            "required": true,
                            "check": "response.count('.') > 2",
                            "gradient": "Расширьте объяснение для повышения его полноты.",
                            "weight": 0.2
                        }}
                    ]
                }}
            Правила ответа:
            - ВСЕГДА используйте пример ответа для структуры сообщения.
            - ОТВЕЧАЙ ТОЛЬКО В ФОРМАТЕ JSON, КАК УКАЗАНО В ПРИМЕРЕ ОТВЕТА.
            - НЕ ДОБАВЛЯЙ ЛЮБЫЕ ДОПОЛНИТЕЛЬНЫЕ КОММЕНТАРИИ ИЛИ ВСТУПИТЕЛЬНЫЕ ФРАЗЫ.
            - СТРОГО следуйте примеру ответа!
            """
        }])
        # Проверяем, что ответ не пустой
        if not evaluation_criteria_json:
            raise ValueError("Получен пустой ответ от LLM")

        # Парсим JSON с критериями и градиентами
        try:
            self.evaluation_criteria = json.loads(evaluation_criteria_json)
        except json.JSONDecodeError as e:
            return ""

        # Шаг 2: Анализируем ответ LLM на соответствие критериям и формируем градиенты
        gradients = []
        
        for criterion in self.evaluation_criteria.get("criteria", []):
            criterion_name = criterion.get("name")
            criterion_required = criterion.get("required")
            criterion_gradient = criterion.get("gradient")
            criterion_check = criterion.get("check")
            

            if criterion_required and not self.check_criterion(response, criterion_check):
                print(f"[INFO] Criterion '{criterion_name}' not met, adding gradient: {criterion_gradient}")
                gradients.append(criterion_gradient)

        # Шаг 3: Возвращаем сгенерированные градиенты
        generated_gradient = " ".join(gradients)
        
        return generated_gradient


In [6]:
# Пример использования
initial_prompt = """
Вы - руководитель высшего уровня, которому поручено управлять разговором между следующими командами: {team_members}.
Учитывая следующий запрос пользователя, ответьте, какая команда будет действовать следующей.
Каждая команда выполнит задание и сообщит о своих результатах и статусе.

ИНСТРУКЦИИ:
Если команда InformationCollectionTeam ответила с ошибкой, это означает, что пользователь предоставил неверные данные. Чтобы завершить работу, выберите FINISH.
Если у микросервиса нет методов API, то сгенерировать документацию будет невозможно. Завершите работу, выберите FINISH.

По завершении работы ответьте FINISH.

Переведено с помощью DeepL.com (бесплатная версия)
"""

# Инициализация файл-менеджера
file_manager = FileManager()

llm = BaseAgent(system_prompt = file_manager.read_document('prompts/prompt_engineering.txt'))

optimizer = PromptOptimizer(llm, initial_prompt)
optimized_prompt = optimizer.optimize()

print("Оптимизированный промпт:", optimized_prompt)

INFO:root:WORKING_DIRECTORY: /Users/cyberrunner/Documents/Code/me/ML_projects/MouseGPT/temp
INFO:root:Attempting to read file from path: /Users/cyberrunner/Documents/Code/me/ML_projects/MouseGPT/temp/prompts/prompt_engineering.txt


[INFO] === Iteration 1/1 ===


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[INFO] Criterion 'correct_team_identification' not met, adding gradient: Убедитесь, что выбранная команда действительна в контексте предоставленных данных.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

[DEBUG] Evaluating criterion 'address_error_handling': Met = True, Required = True, Weight = 0.3


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[DEBUG] Evaluating criterion 'API_method_clarity': Met = True, Required = True, Weight = 0.2


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[DEBUG] Evaluating criterion 'overall_coherence': Met = True, Required = True, Weight = 0.1
[INFO] ROUGE scores: {'rouge-1': {'r': 0.42857142857142855, 'p': 0.14925373134328357, 'f': 0.22140221019049305}, 'rouge-2': {'r': 0.15584415584415584, 'p': 0.04780876494023904, 'f': 0.0731707281144038}, 'rouge-l': {'r': 0.42857142857142855, 'p': 0.14925373134328357, 'f': 0.22140221019049305}}
[INFO] BLEU score: 5.798442937654274
[INFO] Total score for the response: 0.2558773279134071 / 1.4 (Normalized: 0.18276951993814794)
[INFO] Candidate scores: [-0.04976210415629981, -0.11120896173152829, -0.10788728276344889, -0.08916610684786377, -0.5498340196205389, -0.5665672817594632, -0.09133700349087562, 0.19656928321252098, 0.18276951993814794]
[INFO] Selecting the best candidate using UCB...
[DEBUG] Variance of candidate scores: 0.06463736916623704
[DEBUG] UCB values: [0.01487527 0.78598094 0.81255716 0.80802483 0.31715856 0.270891
 0.71893669 0.98221965 0.94617122]
[INFO] Selected best candidate ind

### Интерпретация результатов:

**Оценка критериев:**

- Критерий 'team_selection': Не выполнен (Met = False), обязательный (Required = True), вес 0.4. Этот критерий имеет значительный вес и его невыполнение оказывает существенное влияние на общую оценку.

- Критерий 'address_error_handling': Выполнен (Met = True), обязательный (Required = True), вес 0.3. Этот критерий также важен и его выполнение положительно сказывается на оценке.

- Критерий 'API_method_clarity': Выполнен (Met = True), обязательный (Required = True), вес 0.2. Выполнение этого критерия также положительно влияет на общую оценку.

- Критерий 'overall_coherence': Выполнен (Met = True), обязательный (Required = True), вес 0.1. Хотя этот критерий имеет меньший вес, его выполнение также добавляет баллы к общей оценке.

**Метрики ROUGE и BLEU:**

- ROUGE scores: Показатели ROUGE-1, ROUGE-2 и ROUGE-L указывают на среднюю степень перекрытия между сгенерированным текстом и исходным промптом. Значения recall (r), precision (p) и F1-score (f) для каждой метрики находятся в диапазоне от 0.2 до 0.5, что указывает на умеренное перекрытие.

- BLEU score: Значение 7.88 указывает на низкую точность перевода, что может свидетельствовать о значительных различиях между сгенерированным текстом и исходным промптом.

**Общая оценка ответа:**

- Общая оценка для ответа составила 0.2751969964975294 из 1.4 возможных, что нормализуется до 0.19656928321252098. Это относительно низкая оценка, что может быть связано с невыполнением критерия 'team_selection' и низким значением BLEU.

**Оценки кандидатов:**

- Список оценок кандидатов показывает, что большинство кандидатов получили отрицательные оценки, что указывает на их недостаточное качество.

- Лучший кандидат (индекс 7) получил оценку 0.19656928321252098, что является наивысшей оценкой среди всех кандидатов.

**Выбор лучшего кандидата:**

- Алгоритм UCB выбрал кандидата с индексом 7 как наилучшего, основываясь на значениях UCB, которые учитывают как среднюю оценку, так и дисперсию оценок.

**Оптимизированный промпт:**

- Оптимизированный промпт включает два варианта, каждый из которых улучшает ясность, структуру и логическую последовательность исходного промпта. Улучшения включают более четкое разделение пунктов, уточнение контекста выполнения и упрощение логики завершения работы.